# Instagram Scraper

En el siguiente Notebook se desarrolla un programa con Selenium para hacer Scraping de la red social Instagram y generar una base de datos de publicaciones conformadas por (imagen y leyenda).

In [274]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait

In [291]:
import os
import wget
import time
import pandas as pd

In [276]:
# Crear sesión del navegador y abrir plataforma web de IG
driver = webdriver.Chrome('/Users/gerardosilvarazo')
driver.get('https://www.instagram.com/')

In [277]:
# Seleccionar los inputs de usuario y password
username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='username']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='password']")))

# Vaciar los inputs
username.clear()
password.clear()

# Leer credenciales de archivo
with open('./local/ig_pw.txt', 'r') as f:
    pw = f.read()
    f.close()

# Ingresar las credenciales en la plataforma
username.send_keys("username")
password.send_keys(pw)

In [278]:
# Iniciar sesión en IG
log_in = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()
time.sleep(1)

In [279]:
# Saltar alertas
try:
    not_now = WebDriverWait(driver, 1).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Ahora no')]"))).click()
except:
    pass
try:
    not_now2 = WebDriverWait(driver, 1).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Ahora no')]"))).click()
except:
    pass

In [285]:
# Buscar posts por hashtag
hashtag = "someHashtag"
driver.get("https://www.instagram.com/explore/tags/" + hashtag + "/")

In [286]:
# Saltar el mensaje de obtener ayuda para temas sensibles
try:
    # Mostrar publicaciones
    WebDriverWait(driver, 1).until(EC.element_to_be_clickable((By.XPATH, "//button[@class='sqdOP  L3NKy    _8A5w5    '][contains(text(),'M')]"))).click()
except:
    pass

In [287]:
# Abrir la primer publicación
WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div[class='_9AhH0']"))).click()

In [288]:
# Crear arreglo para almacenar los posts
posts_list = []

In [302]:
# Número de posts a obtener
n_items = 100

# Ciclo que obtiene la imagen y el texto de cada publicación
for i in range(0, n_items):
    print('\n--- POST', i+1)
    post = []
    caption = WebDriverWait(driver, 3).until(EC.visibility_of_element_located((By.XPATH, "//div[@class='C4VMK']/span"))).get_attribute("textContent")
    try:
        image = WebDriverWait(driver, 1).until(EC.visibility_of_element_located((By.XPATH, "//img[@class='FFVAD'][@srcset and not(@srcset='')]"))).get_attribute("src")
        post.append(image)
        post.append(caption)
        print(image)
        print(caption)
        posts_list.append(post)
    except:
        print("Post is a video")
        
    # Pasa al siguiente post
    driver.find_element_by_class_name("coreSpriteRightPaginationArrow").click()



--- POST 1
https://instagram.fntr6-3.fna.fbcdn.net/v/t51.2885-15/e35/226023183_5968579423184381_2767715268821360474_n.jpg?_nc_ht=instagram.fntr6-3.fna.fbcdn.net&_nc_cat=1&_nc_ohc=OQ-CVE-McjcAX_5MIfc&edm=AP_V10EBAAAA&ccb=7-4&oh=6709821217944fe7b5f10628402bcf1c&oe=610A135E&_nc_sid=4f375e
LAMBORGHINI URUS LP650-4⠀⠀2019 Preto / 9.175 KM / R$ 2.890.000,00⠀⠀Motor 4.0 V8 Bi-turbo 650cv⠀Câmbio automático sequencial oito marchas⠀Freio em Cerâmica⠀Faz de 0 a 100km/h em 3,6s⠀Velocidade máxima de 305km/h⠀tração integral permanente 4X4⠀Suspensão esportiva e rodas aro 23”⠀Painel em couro e banco esportivo elétrico⠀Massageador, aquecimento e resfriamento dos bancos dianteiros⠀Teto solar panorâmico em Alcântara⠀2 Telas traseira⠀Câmera de ré 360° , acabamento interno em carbono⠀Escapamento esportivo⠀Leitor de faixa, Head Up Display⠀Carregador de celular por indução⠀Controle cruzeiro adaptativo⠀⠀*Interessados, entre em contato comigo pelo direct ou pelo WhatsApp acessando o link na bio para mais inform

In [303]:
#Convertir el arreglo de publicaciones en un DataFrame
df = pd.DataFrame(posts_list, columns=['image', 'caption'])

# Guarda el DataFrame en archivo .csv
df.to_csv('instagram-posts.csv', index=False)

In [304]:
df

,image,caption
0,https://instagram.fntr6-3.fna.fbcdn.net/v/t51....,"BMW M5, IMPECÁVEL! ANO 2019 COM 33 MIL KM. TO..."
1,https://instagram.fntr6-3.fna.fbcdn.net/v/t51....,🔛 MARQUE UM POSSÍVEL COMPRADOR E GANHE COMISS...
2,https://instagram.fntr6-3.fna.fbcdn.net/v/t51....,🔥😏
3,https://instagram.fntr6-3.fna.fbcdn.net/v/t51....,"À VENDA#jettatsi #jettarline R$ 123.900,00CONT..."
4,https://instagram.fntr6-3.fna.fbcdn.net/v/t51....,LAMBORGHINI URUS LP650-4⠀⠀2019 Preto / 9.175 K...
5,https://instagram.fntr6-3.fna.fbcdn.net/v/t51....,For Sale: BMW 320i 2.0 Turbo Sport GP➖➖➖➖➖➖➖➖➖...
6,https://instagram.fntr6-1.fna.fbcdn.net/v/t51....,MANDE SUA FOTO OU VIDEO 📷Sigam nossa pagina✅Cu...
